# Агрегирование к артикулу автозапчасти применимость по марке+модели автопроизводителя  

Данная обработка предназначена для агрегации и подготовки данных по применимости автозапчасти к различным автомобилям.

Сначала загуржаем исходный файл

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving Teknorot_Catalogue_crosses_AB gn.xlsx to Teknorot_Catalogue_crosses_AB gn.xlsx


## Импортируем нужные библиотеки и создаем датафрейм

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

df = pd.read_excel('Teknorot_Catalogue_crosses_AB gn.xlsx')


Выводим датафрейм на экран, чтобы удбней было ориентироваться в названиях столбцов.

In [ ]:
df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21739 entries, 0 to 21738
Data columns (total 59 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Part No               21739 non-null  object 
 1   PartNo                21739 non-null  object 
 2   Make                  21739 non-null  object 
 3   бренд OE(наш портал)  21739 non-null  object 
 4   Model                 21739 non-null  object 
 5   Bodycode              17875 non-null  object 
 6   Year                  21739 non-null  object 
 7   Year Only             1658 non-null   object 
 8   Description           21739 non-null  object 
 9   Part Position         21739 non-null  object 
 10  Notes                 5747 non-null   object 
 11  QPV                   21739 non-null  int64  
 12  Material              21739 non-null  object 
 13  Axial Length          1284 non-null   float64
 14  Lenght                20168 non-null  float64
 15  Height             

,Part No,PartNo,Make,бренд OE(наш портал),Model,Bodycode,Year,Year Only,Description,Part Position,...,Statu,Paired Article,Ctr,Delphi,Febi,Lemforder,Moog,Moog EU,Trw,Optimal
0,A-100,A100,AUDI,VAG,80,NaN,1972-1978,NaN,Steering Tie Rod End,Front,...,A,-,NaN,TA1132,2234,1014604,NaN,VO-ES-3026,JTE139,G1-105
1,A-100,A100,VOLKSWAGEN,VAG,POLO,86,1974-1994,NaN,Steering Tie Rod End,Front,...,A,-,NaN,TA1132,2234,1014604,NaN,VO-ES-3026,JTE139,G1-105
2,A-1006,A1006,VOLKSWAGEN,VAG,TOUAREG,7P,2010-2017,NaN,Suspension Stabilizer Bar Link,Rear Right,...,A,A-1007,NaN,TC1799,101042,31897 01,NaN,VO-LS-4398,JTS522,G7-1085S
3,A-1006,A1006,VOLKSWAGEN,VAG,TOUAREG,"( 7LA,7L6,7L7 )",2002-2010,NaN,Suspension Stabilizer Bar Link,Rear Right,...,A,A-1007,NaN,TC1799,101042,31897 01,NaN,VO-LS-4398,JTS522,G7-1085S
4,A-1006,A1006,PORSCHE,PORSCHE,CAYENNE,955,2002-2010,NaN,Suspension Stabilizer Bar Link,Rear Right,...,A,A-1007,NaN,TC1799,101042,31897 01,NaN,VO-LS-4398,JTS522,G7-1085S


Добавляем две новые колонки, которые понадобятся для дальнейшей обработки данных:  
1) Колонку с номером первого вхождения набора символов " (" в колонке Model)  
2) Колонку с длиной текста, содержащегося в колонке Model

In [ ]:
df['num_bracket'] = df['Model'].str.find(' (')
df['model_len'] = df['Model'].str.len()
df.loc[df['num_bracket'] == -1,'num_bracket'] = df.loc[df['num_bracket'] == -1,'model_len']
df.head()

,Part No,PartNo,Make,бренд OE(наш портал),Model,Bodycode,Year,Year Only,Description,Part Position,...,Ctr,Delphi,Febi,Lemforder,Moog,Moog EU,Trw,Optimal,num_bracket,model_len
0,A-100,A100,AUDI,VAG,80,NaN,1972-1978,NaN,Steering Tie Rod End,Front,...,NaN,TA1132,2234,1014604,NaN,VO-ES-3026,JTE139,G1-105,NaN,NaN
1,A-100,A100,VOLKSWAGEN,VAG,POLO,86,1974-1994,NaN,Steering Tie Rod End,Front,...,NaN,TA1132,2234,1014604,NaN,VO-ES-3026,JTE139,G1-105,4.0,4.0
2,A-1006,A1006,VOLKSWAGEN,VAG,TOUAREG,7P,2010-2017,NaN,Suspension Stabilizer Bar Link,Rear Right,...,NaN,TC1799,101042,31897 01,NaN,VO-LS-4398,JTS522,G7-1085S,7.0,7.0
3,A-1006,A1006,VOLKSWAGEN,VAG,TOUAREG,"( 7LA,7L6,7L7 )",2002-2010,NaN,Suspension Stabilizer Bar Link,Rear Right,...,NaN,TC1799,101042,31897 01,NaN,VO-LS-4398,JTS522,G7-1085S,7.0,7.0
4,A-1006,A1006,PORSCHE,PORSCHE,CAYENNE,955,2002-2010,NaN,Suspension Stabilizer Bar Link,Rear Right,...,NaN,TC1799,101042,31897 01,NaN,VO-LS-4398,JTS522,G7-1085S,7.0,7.0


Если в колонках model есть данные в скобках ( обычно они лишние и их можно убрать), то удаляем их , используя как раз те две новые колонки из прошлого пункта.

In [ ]:
display(df.head())
for i in (df['num_bracket'].unique()).astype(int):
  df.loc[df['num_bracket'] == i,'Model'] = df.loc[df['num_bracket'] == i,'Model'].str[0:i]

,Part No,PartNo,Make,бренд OE(наш портал),Model,Bodycode,Year,Year Only,Description,Part Position,...,Ctr,Delphi,Febi,Lemforder,Moog,Moog EU,Trw,Optimal,num_bracket,model_len
0,A-100,A100,AUDI,VAG,80,NaN,1972-1978,NaN,Steering Tie Rod End,Front,...,NaN,TA1132,2234,1014604,NaN,VO-ES-3026,JTE139,G1-105,NaN,NaN
1,A-100,A100,VOLKSWAGEN,VAG,POLO,86,1974-1994,NaN,Steering Tie Rod End,Front,...,NaN,TA1132,2234,1014604,NaN,VO-ES-3026,JTE139,G1-105,4.0,4.0
2,A-1006,A1006,VOLKSWAGEN,VAG,TOUAREG,7P,2010-2017,NaN,Suspension Stabilizer Bar Link,Rear Right,...,NaN,TC1799,101042,31897 01,NaN,VO-LS-4398,JTS522,G7-1085S,7.0,7.0
3,A-1006,A1006,VOLKSWAGEN,VAG,TOUAREG,"( 7LA,7L6,7L7 )",2002-2010,NaN,Suspension Stabilizer Bar Link,Rear Right,...,NaN,TC1799,101042,31897 01,NaN,VO-LS-4398,JTS522,G7-1085S,7.0,7.0
4,A-1006,A1006,PORSCHE,PORSCHE,CAYENNE,955,2002-2010,NaN,Suspension Stabilizer Bar Link,Rear Right,...,NaN,TC1799,101042,31897 01,NaN,VO-LS-4398,JTS522,G7-1085S,7.0,7.0


In [ ]:
display(df.head())

,Part No,PartNo,Make,бренд OE(наш портал),Model,Bodycode,Year,Year Only,Description,Part Position,...,Ctr,Delphi,Febi,Lemforder,Moog,Moog EU,Trw,Optimal,num_bracket,model_len
0,A-100,A100,AUDI,VAG,80,NaN,1972-1978,NaN,Steering Tie Rod End,Front,...,NaN,TA1132,2234,1014604,NaN,VO-ES-3026,JTE139,G1-105,NaN,NaN
1,A-100,A100,VOLKSWAGEN,VAG,POLO,86,1974-1994,NaN,Steering Tie Rod End,Front,...,NaN,TA1132,2234,1014604,NaN,VO-ES-3026,JTE139,G1-105,4.0,4.0
2,A-1006,A1006,VOLKSWAGEN,VAG,TOUAREG,7P,2010-2017,NaN,Suspension Stabilizer Bar Link,Rear Right,...,NaN,TC1799,101042,31897 01,NaN,VO-LS-4398,JTS522,G7-1085S,7.0,7.0
3,A-1006,A1006,VOLKSWAGEN,VAG,TOUAREG,"( 7LA,7L6,7L7 )",2002-2010,NaN,Suspension Stabilizer Bar Link,Rear Right,...,NaN,TC1799,101042,31897 01,NaN,VO-LS-4398,JTS522,G7-1085S,7.0,7.0
4,A-1006,A1006,PORSCHE,PORSCHE,CAYENNE,955,2002-2010,NaN,Suspension Stabilizer Bar Link,Rear Right,...,NaN,TC1799,101042,31897 01,NaN,VO-LS-4398,JTS522,G7-1085S,7.0,7.0


Наши конечные данные в том числе будут использоваться для наименования товара. А Озон не любит когда в названии только заглавные буквы, поэтмоу исправим это. Сделаем названия моделей автомобиля более приятными глазу.

In [ ]:
abm_data = df.copy(deep=True)
def capitaliezed_model(model):
  if type(model) == str:
    return model.capitalize()
  else:
    return model

# abm_data = abm_data.drop(['Бренд','Модификация','Год выпуска','Номер TecDoc'],axis=1)
abm_data['Model'] = abm_data['Model'].apply(capitaliezed_model)
abm_data.head(10)

,Part No,PartNo,Make,бренд OE(наш портал),Model,Bodycode,Year,Year Only,Description,Part Position,...,Ctr,Delphi,Febi,Lemforder,Moog,Moog EU,Trw,Optimal,num_bracket,model_len
0,A-100,A100,AUDI,VAG,80,NaN,1972-1978,NaN,Steering Tie Rod End,Front,...,NaN,TA1132,2234,1014604,NaN,VO-ES-3026,JTE139,G1-105,NaN,NaN
1,A-100,A100,VOLKSWAGEN,VAG,Polo,86,1974-1994,NaN,Steering Tie Rod End,Front,...,NaN,TA1132,2234,1014604,NaN,VO-ES-3026,JTE139,G1-105,4.0,4.0
2,A-1006,A1006,VOLKSWAGEN,VAG,Touareg,7P,2010-2017,NaN,Suspension Stabilizer Bar Link,Rear Right,...,NaN,TC1799,101042,31897 01,NaN,VO-LS-4398,JTS522,G7-1085S,7.0,7.0
3,A-1006,A1006,VOLKSWAGEN,VAG,Touareg,"( 7LA,7L6,7L7 )",2002-2010,NaN,Suspension Stabilizer Bar Link,Rear Right,...,NaN,TC1799,101042,31897 01,NaN,VO-LS-4398,JTS522,G7-1085S,7.0,7.0
4,A-1006,A1006,PORSCHE,PORSCHE,Cayenne,955,2002-2010,NaN,Suspension Stabilizer Bar Link,Rear Right,...,NaN,TC1799,101042,31897 01,NaN,VO-LS-4398,JTS522,G7-1085S,7.0,7.0
5,A-1006,A1006,AUDI,VAG,Q7,4L,2006-2015,NaN,Suspension Stabilizer Bar Link,Rear Right,...,NaN,TC1799,101042,31897 01,NaN,VO-LS-4398,JTS522,G7-1085S,2.0,2.0
6,A-1006,A1006,PORSCHE,PORSCHE,Cayenne,92A,2011-2017,NaN,Suspension Stabilizer Bar Link,Rear Right,...,NaN,TC1799,101042,31897 01,NaN,VO-LS-4398,JTS522,G7-1085S,7.0,7.0
7,A-1007,A1007,VOLKSWAGEN,VAG,Touareg,"( 7LA,7L6,7L7 )",2002-2010,NaN,Suspension Stabilizer Bar Link,Rear Left,...,NaN,TC1798,101041,31896 01,NaN,VO-LS-4397,JTS521,G7-1086,7.0,7.0
8,A-1007,A1007,VOLKSWAGEN,VAG,Touareg,7P,2010-2017,NaN,Suspension Stabilizer Bar Link,Rear Left,...,NaN,TC1798,101041,31896 01,NaN,VO-LS-4397,JTS521,G7-1086,7.0,7.0
9,A-1007,A1007,PORSCHE,PORSCHE,Cayenne,92A,2011-2017,NaN,Suspension Stabilizer Bar Link,Rear Left,...,NaN,TC1798,101041,31896 01,NaN,VO-LS-4397,JTS521,G7-1086,7.0,7.0


Теперь наши данные готовы к агрегации. Переходи непосредственно к ним. Важно чтобы у нас бесконечно не повторялся бренд автомобиля. Например AUDI A6 и Q5 должны идти вместе, но при этом Производители автомобилей должны быть разделены ", ".(Смотри вывод ниже). Так же удаляем все другие знаки препинания и заменяем символы типа "É","Ë" на "Е", которые принимает шаблон ОЗОНа.

In [ ]:
# Агрегируем модели с помощью функции set
agg_func_text = {'Model': [set]}
# Создаем таблицу формата "Артикул|Бренд|Агрегированные модели бренда"
abm_data_grouped = abm_data.groupby(['PartNo','Make']).agg(agg_func_text)
# Сбрасываем многоуровневые индексы
abm_data_grouped.columns = abm_data_grouped.columns.droplevel(1)
abm_data_grouped = abm_data_grouped.reset_index()
# Получаем таблицу формата "артикул|бренд + модели бренда"
abm_data_grouped['Model'] = abm_data_grouped['Model'].astype(str)
abm_data_grouped['new_col'] = abm_data_grouped['Make'] + abm_data_grouped['Model']
abm_data_grouped = abm_data_grouped.drop(['Make','Model'],axis=1)
abm_data_grouped.columns = ['Артикул','Применимость']

# Делаем копию полученной таблицы, чтобы избежать ее корректировки.
abm_data_grouped_1 = abm_data_grouped.copy(deep=True)

#агрегируем с помощью функции set ранее полученную колонку, содержащую Бренд+модели бренда
agg_func_text_1 = {'Применимость': [set]}
# Группируем таблицу по принципу "Артикул|Бренд1+Агрегированные модели бренда1, Бренд2+Агрегированные модели бренда2, ..."
# и устраняем мультииндексы.
abm_data_grouped_1.info()
abm_data_grouped_1.head()
abm_data_grouped_1 = abm_data_grouped_1.groupby(['Артикул']).agg(agg_func_text_1)
abm_data_grouped_1.columns = abm_data_grouped_1.columns.droplevel(1)

# Убираем из текста все ненужные символы, а так же заменяем те, которые озон не распознает
abm_data_grouped_1['Применимость'] = abm_data_grouped_1['Применимость'].astype(str)
abm_data_grouped_1['Применимость'] = abm_data_grouped_1['Применимость'].replace(to_replace =['\'','\"'], value = '', regex = True)
abm_data_grouped_1['Применимость'] = abm_data_grouped_1['Применимость'].replace(to_replace =['\{','\}','  ','\/','\/'], value = ' ', regex = True)
abm_data_grouped_1['Применимость'] = abm_data_grouped_1['Применимость'].replace(to_replace =['É','Ë'], value = 'E', regex = True)

abm_data_grouped_1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13491 entries, 0 to 13490
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Артикул       13491 non-null  object
 1   Применимость  13491 non-null  object
dtypes: object(2)
memory usage: 210.9+ KB


,Применимость
Артикул,
A100,"VOLKSWAGEN Polo , AUDI 80"
A1006,"AUDI Q7 , VOLKSWAGEN Touareg , PORSCHE Cayenne"
A1007,"AUDI Q7 , VOLKSWAGEN Touareg , PORSCHE Cayenne"
A10082,"AUDI A6, Q5"
A10092,"AUDI A6, Q5"


## Сохраняем полученный датасет в excel файл и скачиваем.

In [ ]:
abm_data_grouped_1.to_excel('to_use.xlsx', index=True)

In [ ]:
files.download('/content/to_use.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>